<a href="https://colab.research.google.com/github/world779/GCI/blob/master/home_credit_default_risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:

path = '/content/drive/My Drive/GCI2020summer/コンペ第2回/input/'

df = pd.read_csv(path + 'train.csv')
df_test = pd.read_csv(path + 'test.csv')

In [4]:
#車の年数があるときはFLAG_OWN_CARをYESにする
df.loc[(df["OWN_CAR_AGE"].values > 0) & (df["FLAG_OWN_CAR"].isnull()), "FLAG_OWN_CAR"] = "Y"
df.loc[(df["OWN_CAR_AGE"].values == 0) & (df["FLAG_OWN_CAR"].isnull()), "FLAG_OWN_CAR"] = "N"

In [5]:
df_test.loc[(df_test["OWN_CAR_AGE"].values > 0) & (df_test["FLAG_OWN_CAR"].isnull()), "FLAG_OWN_CAR"] = "Y"
df_test.loc[(df_test["OWN_CAR_AGE"].values == 0) & (df_test["FLAG_OWN_CAR"].isnull()), "FLAG_OWN_CAR"] = "N"

In [6]:
print(df.shape)
print(df_test.shape)

(171202, 51)
(61500, 50)


In [7]:
df.isnull().sum()

SK_ID_CURR                          0
TARGET                              0
NAME_CONTRACT_TYPE                  0
CODE_GENDER                         0
FLAG_OWN_CAR                    22675
FLAG_OWN_REALTY                 34241
CNT_CHILDREN                        0
AMT_INCOME_TOTAL                    0
AMT_CREDIT                          0
AMT_ANNUITY                         6
AMT_GOODS_PRICE                   163
NAME_TYPE_SUITE                   747
NAME_INCOME_TYPE                    0
NAME_EDUCATION_TYPE                 0
NAME_FAMILY_STATUS                  0
NAME_HOUSING_TYPE                   0
REGION_POPULATION_RELATIVE          0
DAYS_BIRTH                          0
DAYS_EMPLOYED                       0
DAYS_REGISTRATION                   0
DAYS_ID_PUBLISH                     0
OWN_CAR_AGE                    112992
FLAG_MOBIL                          0
FLAG_EMP_PHONE                      0
FLAG_WORK_PHONE                     0
FLAG_CONT_MOBILE                    0
FLAG_PHONE  

In [8]:
df_test.isnull().sum()

SK_ID_CURR                         0
NAME_CONTRACT_TYPE                 0
CODE_GENDER                        0
FLAG_OWN_CAR                    8138
FLAG_OWN_REALTY                12334
CNT_CHILDREN                       0
AMT_INCOME_TOTAL                   0
AMT_CREDIT                         0
AMT_ANNUITY                        3
AMT_GOODS_PRICE                   53
NAME_TYPE_SUITE                  248
NAME_INCOME_TYPE                   0
NAME_EDUCATION_TYPE                0
NAME_FAMILY_STATUS                 0
NAME_HOUSING_TYPE                  0
REGION_POPULATION_RELATIVE         0
DAYS_BIRTH                         0
DAYS_EMPLOYED                      0
DAYS_REGISTRATION                  0
DAYS_ID_PUBLISH                    0
OWN_CAR_AGE                    40909
FLAG_MOBIL                         0
FLAG_EMP_PHONE                     0
FLAG_WORK_PHONE                    0
FLAG_CONT_MOBILE                   0
FLAG_PHONE                         0
FLAG_EMAIL                         0
O

In [9]:
df.loc[(df["NAME_HOUSING_TYPE"].values == "House / apartment") & (df["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "Y"
df.loc[(df["NAME_HOUSING_TYPE"].values == "Rented apartment") & (df["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "N"
df.loc[(df["NAME_HOUSING_TYPE"].values == "Office apartment") & (df["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "N"
df.loc[(df["NAME_HOUSING_TYPE"].values == "Municipal apartment") & (df["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "N"
df.loc[(df["NAME_HOUSING_TYPE"].values == "Co-op apartment") & (df["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "N"
df.loc[(df["NAME_HOUSING_TYPE"].values == "With parents ") & (df["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "N"

In [10]:
df_test.loc[(df_test["NAME_HOUSING_TYPE"].values == "House / apartment") & (df_test["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "Y"
df_test.loc[(df_test["NAME_HOUSING_TYPE"].values == "Rented apartment") & (df_test["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "N"
df_test.loc[(df_test["NAME_HOUSING_TYPE"].values == "Office apartment") & (df_test["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "N"
df_test.loc[(df_test["NAME_HOUSING_TYPE"].values == "Municipal apartment") & (df_test["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "N"
df_test.loc[(df_test["NAME_HOUSING_TYPE"].values == "Co-op apartment") & (df_test["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "N"
df_test.loc[(df_test["NAME_HOUSING_TYPE"].values == "With parents ") & (df_test["FLAG_OWN_REALTY"].isnull()), "FLAG_OWN_REALTY"] = "N"

In [11]:
print(df.shape)
print(df_test.shape)

(171202, 51)
(61500, 50)


In [12]:
df.isnull().sum()

SK_ID_CURR                          0
TARGET                              0
NAME_CONTRACT_TYPE                  0
CODE_GENDER                         0
FLAG_OWN_CAR                    22675
FLAG_OWN_REALTY                  1656
CNT_CHILDREN                        0
AMT_INCOME_TOTAL                    0
AMT_CREDIT                          0
AMT_ANNUITY                         6
AMT_GOODS_PRICE                   163
NAME_TYPE_SUITE                   747
NAME_INCOME_TYPE                    0
NAME_EDUCATION_TYPE                 0
NAME_FAMILY_STATUS                  0
NAME_HOUSING_TYPE                   0
REGION_POPULATION_RELATIVE          0
DAYS_BIRTH                          0
DAYS_EMPLOYED                       0
DAYS_REGISTRATION                   0
DAYS_ID_PUBLISH                     0
OWN_CAR_AGE                    112992
FLAG_MOBIL                          0
FLAG_EMP_PHONE                      0
FLAG_WORK_PHONE                     0
FLAG_CONT_MOBILE                    0
FLAG_PHONE  

In [13]:
def feat_ext_source(df):
    x1 = df['EXT_SOURCE_1'].fillna(-1) + 1e-1
    x2 = df['EXT_SOURCE_2'].fillna(-1) + 1e-1
    x3 = df['EXT_SOURCE_3'].fillna(-1) + 1e-1
    
    df['EXT_SOURCE_1over2_NAminus1_Add0.1'] = x1/x2
    df['EXT_SOURCE_2over1_NAminus1_Add0.1'] = x2/x1
    df['EXT_SOURCE_1over3_NAminus1_Add0.1'] = x1/x3
    df['EXT_SOURCE_3over1_NAminus1_Add0.1'] = x3/x1
    df['EXT_SOURCE_2over3_NAminus1_Add0.1'] = x2/x3
    df['EXT_SOURCE_3over2_NAminus1_Add0.1'] = x3/x2
    
    df['EXT_SOURCE_na1_2'] = (df['EXT_SOURCE_1'].isnull()) * (df['EXT_SOURCE_2'].fillna(0))
    df['EXT_SOURCE_na1_3'] = (df['EXT_SOURCE_1'].isnull()) * (df['EXT_SOURCE_3'].fillna(0))
    df['EXT_SOURCE_na2_1'] = (df['EXT_SOURCE_2'].isnull()) * (df['EXT_SOURCE_1'].fillna(0))
    df['EXT_SOURCE_na2_3'] = (df['EXT_SOURCE_2'].isnull()) * (df['EXT_SOURCE_3'].fillna(0))
    df['EXT_SOURCE_na3_1'] = (df['EXT_SOURCE_3'].isnull()) * (df['EXT_SOURCE_1'].fillna(0))
    df['EXT_SOURCE_na3_2'] = (df['EXT_SOURCE_3'].isnull()) * (df['EXT_SOURCE_2'].fillna(0))
    
    df['CREDIT_LENGTH'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    
    return df

In [14]:
df = feat_ext_source(df)
df_test  = feat_ext_source(df_test)

In [15]:
def cat_to_dummy(train, test):
    train_d = pd.get_dummies(train, drop_first=False)
    test_d = pd.get_dummies(test, drop_first=False)
    # make sure that the number of features in train and test should be same
    for i in train_d.columns:
        if i not in test_d.columns:
            if i!='TARGET':
                train_d = train_d.drop(i, axis=1)
    for j in test_d.columns:
        if j not in train_d.columns:
            if j!='TARGET':
                test_d = test_d.drop(i, axis=1)
    print('Memory usage of train increases from {:.2f} to {:.2f} MB'.format(train.memory_usage().sum() / 1024**2, 
                                                                            train_d.memory_usage().sum() / 1024**2))
    print('Memory usage of test increases from {:.2f} to {:.2f} MB'.format(test.memory_usage().sum() / 1024**2, 
                                                                            test_d.memory_usage().sum() / 1024**2))
    return train_d, test_d

df, df_test = cat_to_dummy(df, df_test)

Memory usage of train increases from 83.59 to 88.00 MB
Memory usage of test increases from 29.56 to 31.14 MB


In [16]:
df=df.fillna(-999)

In [17]:
df_test=df_test.fillna(-999)

In [18]:
df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

In [19]:
df['CREDIT_INCOME_PERCENT'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
df['ANNUITY_INCOME_PERCENT'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']

In [20]:
df_test['DAYS_EMPLOYED_PERC'] = df_test['DAYS_EMPLOYED'] / df_test['DAYS_BIRTH']
df_test['INCOME_CREDIT_PERC'] = df_test['AMT_INCOME_TOTAL'] / df_test['AMT_CREDIT']
df_test['INCOME_PER_PERSON'] = df_test['AMT_INCOME_TOTAL'] / df_test['CNT_FAM_MEMBERS']
df_test['ANNUITY_INCOME_PERC'] = df_test['AMT_ANNUITY'] / df_test['AMT_INCOME_TOTAL']
df_test['PAYMENT_RATE'] = df_test['AMT_ANNUITY'] / df_test['AMT_CREDIT']

In [21]:
df_test['CREDIT_INCOME_PERCENT'] = df_test['AMT_CREDIT'] / df_test['AMT_INCOME_TOTAL']
df_test['ANNUITY_INCOME_PERCENT'] = df_test['AMT_ANNUITY'] / df_test['AMT_INCOME_TOTAL']

In [22]:
print(df.shape)
print(df_test.shape)

(171202, 175)
(61500, 174)


In [23]:
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df_test[bin_feature], uniques = pd.factorize(df_test[bin_feature])

KeyError: ignored

In [24]:
print(df.shape)
print(df_test.shape)

(171202, 175)
(61500, 174)


In [25]:
df

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,...,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,CREDIT_INCOME_PERCENT,ANNUITY_INCOME_PERCENT
0,0,0,0,112500.0,755190.0,36328.5,675000.0,0.010032,-9233,-878,-333.0,-522,-999.0,1,1,1,1,0,0,2.0,2,2,0,1,1,0,1,1,-999.000000,0.372591,-999.000000,0.0,0.0,0.0,0.0,-292.0,-999.0,-999.0,-999.0,-999.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.095094,0.148969,56250.0,0.322920,0.048105,6.712800,0.322920
1,1,0,0,225000.0,585000.0,16893.0,585000.0,0.008019,-20148,365243,-4469.0,-3436,-999.0,1,0,0,1,0,0,2.0,2,2,0,0,0,0,0,0,-999.000000,0.449567,0.553165,0.0,0.0,0.0,0.0,-617.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-18.128003,0.384615,112500.0,0.075080,0.028877,2.600000,0.075080
2,2,0,0,54000.0,334152.0,18256.5,270000.0,0.004960,-18496,-523,-3640.0,-2050,-999.0,1,1,1,1,1,0,2.0,2,2,0,0,0,0,0,0,-999.000000,0.569503,-999.000000,4.0,0.0,4.0,0.0,-542.0,-999.0,-999.0,-999.0,-999.0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.028276,0.161603,27000.0,0.338083,0.054635,6.188000,0.338083
3,3,0,0,67500.0,152820.0,8901.0,135000.0,0.005002,-24177,365243,-4950.0,-3951,-999.0,1,0,0,1,1,0,1.0,3,3,0,0,0,0,0,0,-999.000000,0.105235,0.767523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-15.107044,0.441696,67500.0,0.131867,0.058245,2.264000,0.131867
4,4,1,0,157500.0,271066.5,21546.0,234000.0,0.006296,-10685,-697,-5101.0,-3226,-999.0,1,1,1,1,0,0,2.0,3,3,0,0,0,0,1,1,0.342344,0.202490,0.669057,0.0,0.0,0.0,0.0,-1243.0,0.0,0.0,0.0,4.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.065232,0.581038,78750.0,0.136800,0.079486,1.721057,0.136800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171197,171197,0,0,83250.0,404325.0,20772.0,337500.0,0.031329,-20529,-3059,-11581.0,-3689,21.0,1,1,1,1,0,0,2.0,2,2,0,0,0,0,1,1,-999.000000,0.404560,0.768808,0.0,0.0,0.0,0.0,-

In [ ]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [ ]:
#df, cat_cols = one_hot_encoder(df)
#df_test, cat_cols = one_hot_encoder(df_test)

In [26]:
df.shape

(171202, 175)

In [27]:
df_test.shape

(61500, 174)

In [28]:
X = df.iloc[:, 2:].values
y = df.iloc[:, 1].values

X_test = df_test.iloc[:, 1:].values

In [31]:
X.shape

(171202, 173)

In [30]:
X_test.shape

(61500, 173)

In [ ]:
X_test

array([[1.00000000e+00, 1.44000000e+05, 9.61146000e+05, ...,
        2.93743094e-02, 6.67462500e+00, 1.96062500e-01],
       [0.00000000e+00, 1.03500000e+05, 2.96280000e+05, ...,
        5.42375456e-02, 2.86260870e+00, 1.55260870e-01],
       [1.00000000e+00, 1.80000000e+05, 1.83694500e+05, ...,
        6.11694961e-02, 1.02052500e+00, 6.24250000e-02],
       ...,
       [0.00000000e+00, 1.44000000e+05, 2.91384000e+05, ...,
        9.17191747e-02, 2.02350000e+00, 1.85593750e-01],
       [1.00000000e+00, 1.93500000e+05, 7.46280000e+05, ...,
        7.91847564e-02, 3.85674419e+00, 3.05395349e-01],
       [0.00000000e+00, 1.35000000e+05, 5.68800000e+05, ...,
        2.88844937e-02, 4.21333333e+00, 1.21700000e-01]])

In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [33]:
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
clf = LGBMClassifier(
            boosting_type='gbdt', class_weight=None, colsample_bytree=0.9234,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               metric='None', min_child_samples=399, min_child_weight=0.1,
               min_split_gain=0.0, n_estimators=5000, n_jobs=4, num_leaves=13,
               objective=None, random_state=42, reg_alpha=2, reg_lambda=5,
               silent=True, subsample=0.855, subsample_for_bin=200000,
               subsample_freq=0
        )
#pipeline=make_pipeline(select,rfc)
clf.fit(X_train, y_train, 
                eval_set= [(X_train, y_train), (X_valid, y_valid)], 
                eval_metric='auc', verbose=100, early_stopping_rounds=100  #30
               )

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.778114	valid_1's auc: 0.754712
[200]	training's auc: 0.795959	valid_1's auc: 0.757539
[300]	training's auc: 0.811015	valid_1's auc: 0.757683
Early stopping, best iteration is:
[279]	training's auc: 0.808094	valid_1's auc: 0.75822


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.9234,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               metric='None', min_child_samples=399, min_child_weight=0.1,
               min_split_gain=0.0, n_estimators=5000, n_jobs=4, num_leaves=13,
               objective=None, random_state=42, reg_alpha=2, reg_lambda=5,
               silent=True, subsample=0.855, subsample_for_bin=200000,
               subsample_freq=0)

In [35]:
path = '/content/drive/My Drive/GCI2020summer/コンペ第2回/'

submission = pd.read_csv(path + 'sample_submission.csv')
submission

,SK_ID_CURR,TARGET
0,171202,0.5
1,171203,0.5
2,171204,0.5
3,171205,0.5
4,171206,0.5
...,...,...
61495,232697,0.5
61496,232698,0.5
61497,232699,0.5
61498,232700,0.5


In [36]:
probabilities = clf.predict_proba(df_test.drop(['SK_ID_CURR'], axis=1))
submission = pd.DataFrame({
    'SK_ID_CURR': df_test['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities]
})

In [37]:
submission

,SK_ID_CURR,TARGET
0,171202,0.022698
1,171203,0.182499
2,171204,0.183106
3,171205,0.077930
4,171206,0.173141
...,...,...
61495,232697,0.136582
61496,232698,0.031423
61497,232699,0.068138
61498,232700,0.178296


In [38]:
submission.to_csv('/content/drive/My Drive/GCI2020summer/コンペ第2回/submission.csv', index=False)